In [3]:
pip install opencv-python

     |████████████████████████████████| 28.3MB 5.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [3]:
img = cv2.imread('trump-donald-5.jpg')

In [4]:
%matplotlib auto
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Using matplotlib backend: Qt5Agg


### Conversion en YUV

In [ ]:
yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
cv2.imwrite("yuv.png", yuv)

In [ ]:
img2 = cv2.imread('yuv.png')
cv2.imshow('image', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Conversion en HSV

In [ ]:
plt.imshow(img)

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imwrite("hsv.png", hsv)

In [ ]:
img3 = cv2.imread('hsv.png')
cv2.imshow('image', img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Resize images

In [ ]:
import glob

images = glob.glob('images/*.jpg')
images

In [ ]:
for i in range(len(images)):
    img4 = cv2.imread(images[i])
    cv2.imshow('image', img4)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    scale_percent = 200 # percent of original size
    width = int(img4.shape[1] * scale_percent / 100)
    height = int(img4.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img4, dim, interpolation = cv2.INTER_AREA)
    
    cv2.imshow("Resized image", resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:

    scale_percent = 200 # percent of original size
    width = int(img4.shape[1] * scale_percent / 100)
    height = int(img4.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img4, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv2.imshow("Resized image", resized)
cv2.waitKey(0)
cv2.destroyAllWindows() 

### Lire vidéo

In [ ]:
cap = cv2.VideoCapture('feu_cheminee.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Mise image en noir et blanc

In [ ]:
originalImage = cv2.imread('trump-donald-5.jpg')
grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
  
(thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
 
cv2.imshow('Black white image', blackAndWhiteImage)
cv2.imshow('Original image',originalImage)
cv2.imshow('Gray image', grayImage)
  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('feu_cheminee.mp4')

while(1):

    # Take each frame
    _, frame = cap.read()

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)

    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    k = cv2.waitKey(25) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()

Correction de Julien

In [2]:
%matplotlib auto
import cv2
import numpy as np
 
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

def rescale_frame_w_h(frame, target_w = 640, target_h = 480):
    dim = (target_w, target_h)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

def norm_int(frame):
    cv2.normalize(frame,frame,0,255,cv2.NORM_MINMAX)
    return frame.astype(int)

    
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('feu_cheminee.mp4',0)

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'FMP4')
fourcc = 0
print(cap.get(3),cap.get(4))
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
#out = cv2.VideoWriter('flipped_flame.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 60, (frame_width,frame_height))
fps = cap.get(cv2.CAP_PROP_FPS)
print("fps ",fps)

# Ouverture d'un flux vidéo en écriture
out = cv2.VideoWriter('reduced_flame_color_segmented.avi',fourcc, int(fps), (frame_width,frame_height))
#out2 = cv2.VideoWriter('reduced_flame_gray.avi',fourcc, fps, (frame_width,frame_height))
#out3 = cv2.VideoWriter('reduced_flame_gradienty.avi',fourcc, fps, (frame_width,frame_height))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")


# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()

  if ret == True:

    # Affichage de la vidéo avant filtrage
    cv2.imshow('Source',frame)
    # Convert to binary
    # Application d'un filtre binaire sur l'intensité du canal Rouge
    ret, binary = cv2.threshold(frame[:,:,2],127,255,cv2.THRESH_BINARY)
    # Affichage du premier filtrage
    cv2.imshow('1st filter',binary)
    # Deuxième filtrage R > G > B pour isoler les flammes
    binary = binary*(frame[:,:,0]<frame[:,:,1])*(frame[:,:,1]<frame[:,:,2])
    # Affichage après 2ème filtre (les fenêtres sont superposées)
    cv2.imshow('All filters',binary)
    binary = cv2.cvtColor(binary,cv2.COLOR_GRAY2BGR)     
    # Ecriture dans le flux de sortie  
    out.write(binary)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
   
  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()
out.release()
#out2.release()
# Closes all the frames
cv2.destroyAllWindows()

OSError: [WinError 193] %1 n’est pas une application Win32 valide